In [1]:
import os

import numpy as np
import pandas as pd


from PySide6.QtWidgets import QApplication, QMainWindow, QTableWidget, QTableWidgetItem, QHeaderView, QMenu, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel
from PySide6.QtCore import QTimer, Slot, Qt, QObject, Signal
from PySide6.QtGui import QAction

from pyvirtualdisplay import Display

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


df = pd.read_csv(os.path.join('.', 'data.csv'))

df.head()

,Unnamed: 0,Product_ID,Durability_Score,Manufacturing_Quality,Price,Customer_Usage_Hours,Warranty_Claims,Satisfaction_Level
0,0,1,88.0,9,466.84,3117.3,1,0
1,1,2,78.0,6,342.98,1730.1,0,0
2,2,3,64.0,8,NaN,5387.5,2,0
3,3,4,92.0,8,432.52,4899.8,3,0
4,4,5,57.0,8,252.25,6955.1,0,0


In [2]:
column_names = df.columns
column_names

Index(['Unnamed: 0', 'Product_ID', 'Durability_Score', 'Manufacturing_Quality',
       'Price', 'Customer_Usage_Hours', 'Warranty_Claims',
       'Satisfaction_Level'],
      dtype='object')

In [3]:

def train_new_classifier(dataset):

    # Separate features and target
    X = dataset.drop(columns=["Satisfaction_Level"])  # Drop target column
    y = dataset["Satisfaction_Level"]

    # Split dataset into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Train a Decision Tree Classifier
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Accuracy on raw dataset: {accuracy:.2f}")
    return accuracy

accuracy = train_new_classifier(df)

Accuracy on raw dataset: 0.93


In [4]:
class MyTable(QTableWidget):

    def __init__(self, initial_data, parent=None):
        super().__init__(parent=parent)
        self.initial_data = initial_data
        
        self.setRowCount(len(self.initial_data))
        self.setColumnCount(len(self.initial_data.columns))
        self.setHorizontalHeaderLabels(self.initial_data.columns)

        self.horizontalHeader().setStretchLastSection(True)  # Stretch the last column
        self.horizontalHeader().setSectionResizeMode(QHeaderView.ResizeMode.Stretch)  # Stretch all columns
        self.verticalHeader().setSectionResizeMode(QHeaderView.ResizeMode.Stretch)  # St

        # Enable right-click context menu
        self.setContextMenuPolicy(Qt.CustomContextMenu)
        self.customContextMenuRequested.connect(self.show_context_menu)


        self.itemChanged.connect(self.on_item_changed)

        for index, entry in initial_data.iterrows():
          
            for inner_index, (col_name, item) in enumerate(entry.items()):
                   self.setItem(index, inner_index, QTableWidgetItem(f'{item}'))


    def show_context_menu(self, pos):
        menu = QMenu(self)
        
        # Add actions for deleting rows and columns
        delete_row_action = QAction("Delete Row", self)
        delete_row_action.triggered.connect(self.delete_selected_row)
        menu.addAction(delete_row_action)
        
        delete_column_action = QAction("Delete Column", self)
        delete_column_action.triggered.connect(self.delete_selected_column)
        menu.addAction(delete_column_action)
        
        # Show the menu at the cursor position
        menu.exec(self.mapToGlobal(pos))
    
    def delete_selected_row(self):
        # Get the selected row
        selected_items = self.selectedItems()
      
        if selected_items:
            row = selected_items[0].row()
            self.initial_data.drop(index=row)
            self.removeRow(row)
    
    def delete_selected_column(self):
        # Get the selected column
        selected_items = self.selectedItems()
        if selected_items:
            column = selected_items[0].column()
            column_name = self.initial_data.columns[column]
            print(column_name)
            self.initial_data.pop(column_name)
            self.removeColumn(column)

    @Slot()
    def on_item_changed(self, item):       
         old_entry = df.iloc[item.row(), item.column()]
         
         if f'{old_entry}' == f'{item.text()}':
          
             return

         try:
            item =   float(item)
         except TypeError as e:
             pass
            
         self.initial_data.iloc[item.row(), item.column()] = item.text()


class MyMainWindow(QMainWindow):

    def __init__(self, signal, parent=None):
        super().__init__(parent=parent)
        self.close_signal = signal


    def closeEvent(self, event):
        self.signal.trigger_signal()

        event.accept()


class EndSignal(QObject):

    signal = Signal()


    def __init__(self, name, parent = None):
        super().__init__(parent)
        self.name = name


    def trigger_signal(self):
        self.signal.emit()

        

In [5]:
class View:

    def get_new_acc(self, df):
        self.accuracy = train_new_classifier(df)
        self.text_label.setText(f'Gegenwärtige korrekte Vorhersagen {self.accuracy:.2f}')

    def __init__(self, initial_data, acc):
        self.accuracy = acc
        self.display = Display(visible=0, size=(800, 600))

        self.display.start()

        self.end_signal = EndSignal('End')
        
        self.initial_data = initial_data

        self.app = QApplication([])

        main_window = MyMainWindow(self.end_signal)
        self.end_signal.signal.connect(self.quit)

        self.table = MyTable(self.initial_data)

        central_widget = QWidget()
        main_window.setCentralWidget(central_widget)

        main_layout = QVBoxLayout()
        central_widget.setLayout(main_layout)

        main_layout.addWidget(self.table)

        sub_layout = QHBoxLayout()
        main_layout.addLayout(sub_layout)
       
        self.text_label = QLabel(f'Gegenwärtige korrekte Vorhersagen {self.accuracy:.2f}')
        sub_layout.addWidget(self.text_label)
        button = QPushButton('Modell neu trainieren')
        sub_layout.addWidget(button)
        button.clicked.connect(lambda: self.get_new_acc(self.table.initial_data))
        
        timer = QTimer()

        timer.start(100)

        timer.timeout.connect(self.mainloop)


        main_window.show()
        self.app.exec()


    
    @Slot()
    def quit(self):
        self.display.stop()
        self.app.quit()



    @Slot(QTimer)
    def mainloop(timer):
        pass

   
        
         


View(df, accuracy)

FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden